In [ ]:
# @title Requirements
!pip install roboflow ultralytics torch torchvision tensorflow pillow numpy opencv-python flask pyngrok

In [ ]:
# @title Imports
import os
from pathlib import Path
import cv2
import numpy as np
from zipfile import ZipFile
import shutil
import torch
import tensorflow as tf
from ultralytics import YOLO
from roboflow import Roboflow
from flask import Flask, request, jsonify
from pyngrok import ngrok
from google.colab import drive, userdata
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

os.chdir('/content')
OUTPUT_DIR = "/content/models"
DATASET_YOLO = "/content/dataset_yolo"
DATASET_CNN = "/content/dataset_cnn"
os.makedirs(OUTPUT_DIR, exist_ok=True)

YOLO_FILE_ID = "1D9alUhw0tQ9Z9vJru22H0NPPw7YZ3_PU"
CNN_FILE_ID = "1R4wd9tzUvJVR8dqGY9zlyIzHRS93Q4i3"

if not os.path.exists(DATASET_YOLO):
    !gdown --id {YOLO_FILE_ID} -O /content/dataset_yolo.zip
    with ZipFile("/content/dataset_yolo.zip", 'r') as zip_ref:
        zip_ref.extractall("/content")
    
    extracted_items = [d for d in os.listdir("/content") if d not in ["models", "sample_data", ".config"]]
    for item in extracted_items:
        item_path = f"/content/{item}"
        if os.path.isdir(item_path) and item != "dataset_yolo":
            if os.path.exists(f"{item_path}/data.yaml") or os.path.exists(f"{item_path}/train"):
                shutil.move(item_path, DATASET_YOLO)
                break

if os.path.exists(DATASET_YOLO) and not os.path.exists(f"{DATASET_YOLO}/valid"):
    if os.path.exists(f"{DATASET_YOLO}/test"):
        os.symlink(f"{DATASET_YOLO}/test", f"{DATASET_YOLO}/valid")

if not os.path.exists(DATASET_CNN):
    !gdown --id {CNN_FILE_ID} -O /content/dataset_cnn.zip
    with ZipFile("/content/dataset_cnn.zip", 'r') as zip_ref:
        zip_ref.extractall("/content")
    
    extracted_items = [d for d in os.listdir("/content") if d not in ["models", "dataset_yolo", "sample_data", ".config"]]
    for item in extracted_items:
        item_path = f"/content/{item}"
        if os.path.isdir(item_path) and item != "dataset_cnn":
            if os.path.exists(f"{item_path}/authentic") or os.path.exists(f"{item_path}/counterfeit"):
                shutil.move(item_path, DATASET_CNN)
                break

In [ ]:
# @title YOLO Training
model = YOLO("yolov8n.pt")

results = model.train(
    data=f"{DATASET_YOLO}/data.yaml",
    epochs=50,
    imgsz=640,
    batch=16,
    name="medicine_yolo",
    project=OUTPUT_DIR
)

model.save(f"{OUTPUT_DIR}/medicine_yolo.pt")

metrics = model.val()
print(f"mAP50: {metrics.box.map50}")
print(f"mAP50-95: {metrics.box.map}")

In [ ]:
# @title CNN Dataset setup
os.makedirs(f"{DATASET_CNN}/authentic", exist_ok=True)
os.makedirs(f"{DATASET_CNN}/counterfeit", exist_ok=True)

def crop_and_save(split="train"):
    images_dir = f"{DATASET_YOLO}/{split}/images"
    labels_dir = f"{DATASET_YOLO}/{split}/labels"

    for label_file in Path(labels_dir).glob("*.txt"):
        img_file = Path(images_dir) / f"{label_file.stem}.jpg"

        if not img_file.exists():
            continue

        img = cv2.imread(str(img_file))
        h, w = img.shape[:2]

        with open(label_file, "r") as f:
            for idx, line in enumerate(f.readlines()):
                parts = line.strip().split()
                class_id = int(parts[0])
                x_center, y_center, width, height = map(float, parts[1:])

                x1 = int((x_center - width/2) * w)
                y1 = int((y_center - height/2) * h)
                x2 = int((x_center + width/2) * w)
                y2 = int((y_center + height/2) * h)

                x1, y1 = max(0, x1), max(0, y1)
                x2, y2 = min(w, x2), min(h, y2)

                crop = img[y1:y2, x1:x2]

                class_name = "authentic" if class_id == 0 else "counterfeit"
                output_path = f"{DATASET_CNN}/{class_name}/{label_file.stem}_{idx}.jpg"
                cv2.imwrite(output_path, crop)

crop_and_save("train")
crop_and_save("valid")
crop_and_save("test")

print(f"Authentic crops: {len(list(Path(f'{DATASET_CNN}/authentic').glob('*.jpg')))}")
print(f"Counterfeit crops: {len(list(Path(f'{DATASET_CNN}/counterfeit').glob('*.jpg')))}")

In [ ]:
# @title CNN Training Setup
from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy('mixed_float16')

IMG_SIZE = 160
BATCH_SIZE = 256

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    validation_split=0.1
)

train_generator = train_datagen.flow_from_directory(
    DATASET_CNN,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    subset="training",
    shuffle=True,
    interpolation='nearest'
)

val_generator = train_datagen.flow_from_directory(
    DATASET_CNN,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    subset="validation",
    interpolation='nearest'
)

base_model = MobileNetV2(weights="imagenet", include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))
base_model.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation="relu")(x)
predictions = Dense(2, activation="softmax", dtype='float32')(x)

model_cnn = Model(inputs=base_model.input, outputs=predictions)

In [ ]:
# @title CNN Training
model_cnn.compile(
    optimizer="adam",
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

history = model_cnn.fit(
    train_generator,
    validation_data=val_generator,
    epochs=20
)
model_cnn.save(f"{OUTPUT_DIR}/medicine_cnn.h5")

In [ ]:
# @title Server
NGROK_AUTHTOKEN = userdata.get('NGROK_AUTHTOKEN')
ngrok.set_auth_token(NGROK_AUTHTOKEN)
app = Flask(__name__)

YOLO_MODEL_PATH = f"{OUTPUT_DIR}/medicine_yolo.pt"
CNN_MODEL_PATH = f"{OUTPUT_DIR}/medicine_cnn.h5"

yolo_model = None
cnn_model = None

def load_models():
    global yolo_model, cnn_model
    if os.path.exists(YOLO_MODEL_PATH):
        yolo_model = YOLO(YOLO_MODEL_PATH)
        print("YOLO model loaded")
    if os.path.exists(CNN_MODEL_PATH):
        cnn_model = tf.keras.models.load_model(CNN_MODEL_PATH)
        print("CNN model loaded")

def verify_image(image_array):
    if yolo_model is None or cnn_model is None:
        return {
            "status": "error",
            "message": "Models not loaded",
            "result": None,
            "confidence": 0.0,
            "detections": []
        }

    results = yolo_model.predict(image_array, verbose=False)

    if len(results[0].boxes) == 0:
        return {
            "status": "success",
            "message": "No packages detected",
            "result": None,
            "confidence": 0.0,
            "detections": []
        }

    detections = []

    for box in results[0].boxes:
        x1, y1, x2, y2 = map(int, box.xyxy[0].tolist())
        yolo_class = int(box.cls[0])
        yolo_conf = float(box.conf[0])
        yolo_label = "authentic" if yolo_class == 0 else "counterfeit"

        crop = image_array[y1:y2, x1:x2]
        crop_resized = cv2.resize(crop, (IMG_SIZE, IMG_SIZE))
        crop_array = crop_resized.astype("float32") / 255.0
        crop_array = np.expand_dims(crop_array, axis=0)

        cnn_pred = cnn_model.predict(crop_array, verbose=0)
        cnn_class = np.argmax(cnn_pred[0])
        cnn_conf = float(np.max(cnn_pred[0]))
        cnn_label = "authentic" if cnn_class == 0 else "counterfeit"

        if yolo_label == cnn_label and cnn_conf > 0.8:
            final_result = "GENUINE" if yolo_label == "authentic" else "COUNTERFEIT"
        else:
            final_result = "SUSPICIOUS"

        detections.append({
            "bbox": [x1, y1, x2, y2],
            "yolo_label": yolo_label,
            "yolo_confidence": yolo_conf,
            "cnn_label": cnn_label,
            "cnn_confidence": cnn_conf,
            "result": final_result
        })

    return {
        "status": "success",
        "message": "Verification complete",
        "result": detections[0]["result"] if len(detections) == 1 else "MULTIPLE_PACKAGES",
        "confidence": detections[0]["cnn_confidence"] if len(detections) == 1 else 0.0,
        "detections": detections
    }

@app.route("/", methods=["GET"])
def hello():
    return jsonify({
        "status": "success",
        "message": "Medicine AI Vision Server is running",
        "service": "vision-inspection"
    })


@app.route("/health", methods=["GET"])
def health():
    return jsonify({
        "status": "healthy",
        "service": "vision-inspection"
    })


@app.route("/api/verify", methods=["POST"])
def verify_package():
    if "image" not in request.files:
        return jsonify({
            "status": "error",
            "message": "No image provided"
        }), 400

    image_file = request.files["image"]
    image_bytes = image_file.read()
    nparr = np.frombuffer(image_bytes, np.uint8)
    image_array = cv2.imdecode(nparr, cv2.IMREAD_COLOR)

    if image_array is None:
        return jsonify({
            "status": "error",
            "message": "Invalid image format"
        }), 400

    result = verify_image(image_array)
    return jsonify(result)


if __name__ == "__main__":
    port = 5000

    load_models()

    public_url = ngrok.connect(port)
    print(f"Ngrok Tunnel URL: {public_url}")
    print(f"Add this URL to your Django .env file as COLAB_API_URL")

    app.run(port=port)

In [ ]:
# @title Test Verification
import matplotlib.pyplot as plt
import matplotlib.patches as patches

yolo_model = YOLO(f"{OUTPUT_DIR}/medicine_yolo.pt")
cnn_model = tf.keras.models.load_model(f"{OUTPUT_DIR}/medicine_cnn.h5")

test_images_dir = Path(f"{DATASET_YOLO}/test/images")
test_images = list(test_images_dir.glob("*.jpg"))

if len(test_images) == 0:
    print("No test images found!")
else:
    test_image_path = test_images[0]
    print(f"Testing: {test_image_path.name}\n")
    
    image = cv2.imread(str(test_image_path))
    
    if image is None:
        print("Failed to load image!")
    else:
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        yolo_results = yolo_model.predict(image, verbose=False)
        
        if len(yolo_results[0].boxes) == 0:
            plt.figure(figsize=(10, 10))
            plt.imshow(image_rgb)
            plt.axis('off')
            plt.title("No packages detected", fontsize=14)
            plt.show()
        else:
            fig, ax = plt.subplots(1, figsize=(12, 12))
            ax.imshow(image_rgb)
            
            for idx, box in enumerate(yolo_results[0].boxes):
                x1, y1, x2, y2 = map(int, box.xyxy[0].tolist())
                yolo_class = int(box.cls[0])
                yolo_conf = float(box.conf[0])
                yolo_label = "authentic" if yolo_class == 0 else "counterfeit"
                
                crop = image[y1:y2, x1:x2]
                crop_resized = cv2.resize(crop, (IMG_SIZE, IMG_SIZE))
                crop_array = crop_resized.astype("float32") / 255.0
                crop_array = np.expand_dims(crop_array, axis=0)
                
                cnn_pred = cnn_model.predict(crop_array, verbose=0)
                cnn_class = np.argmax(cnn_pred[0])
                cnn_conf = float(np.max(cnn_pred[0]))
                cnn_label = "authentic" if cnn_class == 0 else "counterfeit"
                
                if yolo_label == cnn_label and cnn_conf > 0.8:
                    final_result = "GENUINE" if yolo_label == "authentic" else "COUNTERFEIT"
                    color = 'green' if final_result == "GENUINE" else 'red'
                else:
                    final_result = "SUSPICIOUS"
                    color = 'orange'
                
                rect = patches.Rectangle((x1, y1), x2-x1, y2-y1, linewidth=3, edgecolor=color, facecolor='none')
                ax.add_patch(rect)
                
                label_text = f"{final_result}\nYOLO: {yolo_label} ({yolo_conf:.2%})\nCNN: {cnn_label} ({cnn_conf:.2%})"
                ax.text(x1, y1-10, label_text, color='white', fontsize=10, 
                       bbox=dict(boxstyle='round', facecolor=color, alpha=0.8))
            
            ax.axis('off')
            plt.tight_layout()
            plt.show()